Video Classification Assignment
CS 200 ML by Arham Khan

In [4]:
#import packages
import cv2     
import math   
import matplotlib.pyplot as plt    
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   
import numpy as np    
from keras.utils import np_utils
from skimage.transform import resize  
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

2023-05-02 15:03:37.796925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Preprocessing happens first. Before that, we create a similar dataframe for test videos

In [5]:
# open file with video names
f = open("/Users/khana/Downloads/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe with video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
print(train.head())

                                    video_name
0  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4  ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


Adding the tag of each video for training and testing sets

In [19]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for testing videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

extracting frames from the training videos which will be used for training the model

In [6]:
# storing frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('/Users/kenfritzell/Desktop/Videos/UCF-101/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing frames in a new folder named train_1
            filename ='/Users/khana/Desktop/train_1/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

  0%|                                          | 1/3783 [00:00<18:15,  3.45it/s]OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g01_c02.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g01_c03.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g01_c04.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g01_c05.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g01_c06.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g02_c01.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_ApplyEyeMakeup_g02_c02.avi"
OpenCV: Couldn't read video stream from file "/Users/kenfritzell/Desktop/Videos/UCF-101/v_Appl

After frame extraction, we save name of these frames with their tag in .csv file

In [7]:
# getting names of all images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating image name
    train_image.append(images[i].split('/')[1])
    # creating class of image
    train_class.append(images[i].split('/')[1].split('_')[1])
    
# storing images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting dataframe into csv file 
train_data.to_csv('UCF/train_new.csv',header=True, index=False)

0it [00:00, ?it/s]


OSError: Cannot save file into a non-existent directory: 'UCF'

importing the libraries first, then extracting frames

In [ ]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
Reading in the training file:

In [ ]:
train = pd.read_csv('UCF/train_new.csv')
train.head()

using the .csv file, we read the frames that we extracted and store as a NumPy array:

In [ ]:
# creating empty list
train_image = []

# for loop for reading and storing frames
for i in tqdm(range(train.shape[0])):
    # loading image and keeping target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting to array
    img = image.img_to_array(img)
    # normalizing pixel value
    img = img/255
    # appending image to the train_image list
    train_image.append(img)
    
# converting list to numpy array
X = np.array(train_image)

# shape of array
X.shape

Output: (73844, 224, 224, 3)

We have 73,844 images each of size (224, 224, 3). Next creating the validation set:

In [ ]:
# separating target
y = train['class']

# creating training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

There are 101 categories for video classification. We will create 101 different columns in the target, one for each category using get_dummies() function.

In [ ]:
# creating dummies of target variable for training and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

Using VGG-16 pre-trained model

In [ ]:
# creating base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

Extracting features from pre-trained model for training and validation image

In [ ]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

Output: (59075, 7, 7, 512)

We have 59,075 images in the training set and the shape has been changed to (7, 7, 512) since we have passed these images through the VGG16 architecture.

Extracting features for validation frames:

In [ ]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

Output: (14769, 7, 7, 512)

There are 14,769 images in the validation set and the shape of these images has also changed to (7, 7, 512).

Reshaping the images into a single dimension and normalizing :

In [ ]:
# reshaping training as well as validation frames in single dimension
X_train = X_train.reshape(59075, 7*7*512)
X_test = X_test.reshape(14769, 7*7*512)

# normalizing pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

Checking the shape of our images:

In [ ]:
# shape of images
X_train.shape

Output: (59075, 25088)

The input shape will be 25,088. Creating the architecture:

In [ ]:
#defining model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

Defineing a function to save the weights of the model:

In [ ]:
# defining function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
Compiling the model:

In [ ]:
# compiling model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

Training the model:

In [ ]:
# training model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)


# Evaluating model

importing evaluation methods:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

Defining model architecture, similar to what we had while training the model

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False)
#defining model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))

loading the trained weights stored as weights.hdf5:

In [ ]:
# loading trained weights
model.load_weights("weights.hdf5")

Compiling model

In [ ]:
# compiling model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Creating test data

In [ ]:
# getting test list
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

Mapping predicted categories with actual categories

In [ ]:
# creating tags
train = pd.read_csv('UCF/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

Generating predictions:

In [ ]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop for extracting frames from test videos
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    #capturing video from given path
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1]) 
    #frame rate
    frameRate = cap.get(5)
    x=1
    # removing all other files from temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        #frame number
        frameId = cap.get(1) 
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing frames of this video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending mode of predictions in predict list to assign tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])

# Evaluating model

In [ ]:
# checking accuracy of predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

In [ ]:
Output: 44.80570975416337

Great! Our model’s accuracy of 44.8% 